In [2]:
import requests
from bs4 import BeautifulSoup
from sklearn.metrics.pairwise import cosine_similarity

url = 'https://raw.githubusercontent.com/skupriienko/Ukrainian-Stopwords/refs/heads/master/stopwords_ua.txt'
# за посиланням знаходиться список українських стоп-слів, який будемо парсити

response = requests.get(url)

if response.status_code == 200:

    soup = BeautifulSoup(response.text, 'html.parser')
    stopwords_text = soup.get_text()
    stopwords = stopwords_text.split('\n')

In [3]:
from datasets import load_dataset

# завантаження датасету українських новин

ds = load_dataset("FIdo-AI/ua-news")

In [4]:
# використання перших 25000 статей із датасету

list_words = ds['train'][:25000]['text']

In [5]:
import re

def remove_stopwords(text, stop_words):
    
    # розбиваємо текст на слова
    
    words = text.split()
    
    # видаляємо апострофи, з якими виникають проблеми при обробці
    
    words = [word.lower().replace("'", "") for word in words] 
    
    # видаляємо стоп-слова та набори символів, які не містять літери
    
    filtered_words = [word for word in words if word not in stop_words and re.search(r'[a-zA-Zа-яА-Я]', word)]
    
    # повертаємо список слів після видалення стоп-слів
    
    return ' '.join(filtered_words)

# запуск видалення стоп-слів із текстів

filtered_texts = [remove_stopwords(text, stopwords) for text in list_words]

In [6]:
print(filtered_texts[999])

# приклад тексту після очищення

прес-секретар канцлера штеффен зайберт, повідомляє reuters. "канцлер жалем прийняла відома заяву відставку премєр-міністра італії. великою довірою працювала маттео ренці але, звичайно, демократичне рішення італійських виборців поважати, рішення премєр-міністра", зазначив зайберт. "німецький уряд пропонує тісну співпрацю партнерство наступному уряду італії, був", додав він. повідомлялося, премєр-міністр італії маттео ренці визнав поразку референдумі парламентської реформи оголосив відставку.


In [7]:
import re
from uk_stemmer import UkStemmer

stemmer = UkStemmer()
stemmed = []
# ініціалізуємо стеммер та створюємо список для речень після стеммінгу

def stem_text(text):
    # розбиваємо текст на список слів за допомогою регулярного виразу
    words = re.split(r'(\W)', text)
    
    # очищення списку від пустих слів
    
    words = [word for word in words if word != '']
    
    # застосування стеммінгу до всіх слів
    
    for i in range(len(words)):
        words[i] = stemmer.stem_word(words[i])
        
    # трансформуємо список слів назад до тексту
    stemmed_text = ''.join(words)
    return stemmed_text

# заповнюємо список стеммінгованих текстів

for i in filtered_texts:
    stemmed.append(stem_text(i))


In [19]:
import gensim
from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel

# видалення слів, що складаються з менш ніж 3 символів, токенізація тексту

def preprocess(text):
    return [word for word in gensim.utils.simple_preprocess(text, min_len = 3)]

# підготовка даних до запуску
processed_docs = [preprocess(doc) for doc in stemmed]

# створення словнику на базі текстів

dictionary = corpora.Dictionary(processed_docs)

# створення корпусу на базі текстів

corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# застосування LDA з 40 темами
lda_model = gensim.models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=40, random_state=88, passes=10)

# отримання слів, які зустрічаються у схожих текстах
# for idx, topic in lda_model.print_topics(-1):
    # print(f"Тема {idx}: {topic}")

coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Coherence Score:  0.5944099204462898


In [20]:
coherence_model_umass = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='u_mass')
coherence_umass = coherence_model_umass.get_coherence()
print('\nCoherence Score: ', coherence_umass)


Coherence Score:  -3.4395374583547764


In [21]:
top_topics = lda_model.top_topics(corpus)


In [22]:
top_topics

[([(0.033947624, 'что'),
   (0.030072985, 'это'),
   (0.021733548, 'год'),
   (0.021061352, 'как'),
   (0.014263, 'украин'),
   (0.013245423, 'которы'),
   (0.0099894125, 'работ'),
   (0.009721731, 'будет'),
   (0.009230642, 'есл'),
   (0.008666335, 'так'),
   (0.008053393, 'тольк'),
   (0.00756852, 'ест'),
   (0.007503645, 'дел'),
   (0.007466628, 'компан'),
   (0.0072481437, 'бизнес'),
   (0.0071074734, 'сво'),
   (0.0066372063, 'больш'),
   (0.0065457006, 'limited'),
   (0.0062491344, 'решен'),
   (0.006244987, 'было')],
  -1.5383011534308015),
 ([(0.02302135, 'збро'),
   (0.022069165, 'обстріл'),
   (0.019455053, 'сил'),
   (0.016778324, 'гранатомет'),
   (0.015742809, 'українськ'),
   (0.015100042, 'район'),
   (0.014979396, 'бойовик'),
   (0.014726714, 'вогн'),
   (0.014497148, 'міномет'),
   (0.014186028, 'вогон'),
   (0.013894927, 'позиці'),
   (0.0134932455, 'стрілецьк'),
   (0.012847237, 'напрямк'),
   (0.011845685, 'пункт'),
   (0.010956218, 'противник'),
   (0.010741532, 'д

In [23]:
# отримуємо розподіл тем для кожного документа в корпусі

doc_topic_distributions = [lda_model.get_document_topics(bow) for bow in corpus]

# функція для перетворення розподілу тем на вектор

def get_topic_vector (doc_topic_dist, num_topics):
    vec = [0] * num_topics
    
    # проходимо по кожній парі (індекс теми, ймовірність)
    
    for topic_idx, prob in doc_topic_dist:
        
        # встановлюємо ймовірність теми в відповідну позицію вектора
        
        vec[topic_idx] = prob
    return vec

In [24]:
# знаходимо для кожного документа вектор ймовірностей тем

topic_vectors = [get_topic_vector(doc, num_topics=40) for doc in doc_topic_distributions]

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

# будуємо матрицю подібності, використовуючи cosine_similarity

similarity_matrix = cosine_similarity(topic_vectors)

In [26]:
import numpy as np

# функція для знаходження найбільш схожого тексту за індексом

def find_most_similar_for_given_text(similarity_matrix, text_index):
    similarities = similarity_matrix[text_index]
    
    # встановлюємо схожість тексту з самим собою на -1, щоб виключити з розгляду
    
    similarities[text_index] = -1
    # знаходимо індекс тексту з найбільшою семантичною схожістю
    most_similar_index = np.argmax(similarities)
    
    return most_similar_index

# обираємо текст, на який будемо шукати схожий

text_index = 21000
print(ds['train'][text_index]['text'])

most_similar_text_index = find_most_similar_for_given_text(similarity_matrix, text_index)

# знаходимо номер схожого тексту та виводимо сам текст

print(most_similar_text_index)
print(ds['train'][int(most_similar_text_index)]['text'])

 Випереджальне падіння імпорту дозволить Україні поліпшити зовнішньоторговельний баланс на $ 4 млрд.  Альфа-Банк Україна погіршив прогноз щодо динаміки українського ВВП, змінивши очікування зростання на 3,2% до падіння на 2%. Про це повідомляється в оновленому макропрогнозі банку.  За оцінками аналітиків, найбільш складним для української економіки стане другий квартал 2020 року. Падіння ВВП в цей період очікується на рівні 7%. У третьому кварталі спад продовжиться, а повноцінно відновитися від шоку економіка зможе лише в 2021 році. Прогноз банку передбачає досягнення Україною угоди про нову кредитну програму з Міжнародним валютним фондом у другому кварталі цього року. Транш МВФ разом із засобами інших міжнародних фінансових організацій допоможе Україні профінансувати основну частину різко збільшеного дефіциту державних фінансів, який досягне 8% ВВП в 2020 році, відзначають укладачі прогнозу. «Через різке розширення бюджетного дефіциту відношення державного боргу до ВВП України в 2020 